In [1]:
from ema_workbench import (ema_logging, RealParameter, IntegerParameter,
                           TimeSeriesOutcome, ScalarOutcome, Constant, MultiprocessingEvaluator, perform_experiments, save_results, load_results)

from integrated_model import IntegratedModel
from integrated_model import mean_over_replications
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mpld3
import operator
from ema_workbench.analysis import prim
from ema_workbench.analysis.plotting_util import VIOLIN, KDE
from ema_workbench.analysis.plotting import lines, envelopes

from ema_workbench.analysis.pairs_plotting import pairs_lines, pairs_scatter, pairs_density
from ema_workbench.util import load_results, ema_logging

import seaborn as sns
ema_logging.LOG_FORMAT = '[%(name)s/%(levelname)s/%(processName)s] %(message)s'
ema_logging.log_to_stderr(ema_logging.INFO)


C:\Anaconda3\lib\site-packages\ema_workbench\em_framework\optimization.py:22: ImportWarning: platypus based optimization not available
  warnings.warn("platypus based optimization not available", ImportWarning)
C:\Anaconda3\lib\site-packages\ema_workbench\connectors\__init__.py:8: ImportWarning: vensim connector not available
  warnings.warn("vensim connector not available", ImportWarning)
C:\Anaconda3\lib\site-packages\ema_workbench\connectors\__init__.py:23: ImportWarning: pysd connector not available
  warnings.warn("pysd connector not available", ImportWarning)
C:\Anaconda3\lib\importlib\_bootstrap.py:205: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


<Logger EMA (DEBUG)>

In [2]:
ema_logging.LOG_FORMAT = '[%(name)s/%(levelname)s/%(processName)s] %(message)s'
ema_logging.log_to_stderr(ema_logging.INFO)

model = IntegratedModel('Model_ecosystem', wd='./model') 
model.replications = 50

#specify uncertainties
model.uncertainties = [RealParameter("energy_demand_growth_f",-0.006, 0.013),
                            RealParameter("a",0,0.2),
                            RealParameter("market_share_base_growth_f", -0.02, 0.02),
                            IntegerParameter("u", 0,39),
                            RealParameter("c",0.03,0.733),
                            RealParameter("other_grey_share_growth_f", -0.015,0.015)
                          ]
#specify outcomes 
model.outcomes = [TimeSeriesOutcome('area_capacity_current', function=mean_over_replications),    
                    TimeSeriesOutcome('area_capacity_unused_current', function=mean_over_replications),           
                    TimeSeriesOutcome('area_TP_current', function=mean_over_replications),                       
                    TimeSeriesOutcome('area_occupancy_current', function=mean_over_replications),                
                    TimeSeriesOutcome('area_denied_infra_cap_current', function=mean_over_replications),          
                    TimeSeriesOutcome('area_denied_cap_percentage_current', function=mean_over_replications),     
                    TimeSeriesOutcome('terminal_capacity_current', function=mean_over_replications),              
                    TimeSeriesOutcome('terminal_capacity_unused_current', function=mean_over_replications),       
                    TimeSeriesOutcome('terminal_TP_current', function=mean_over_replications),                    
                    TimeSeriesOutcome('terminal_occupancy_current', function = mean_over_replications),             
                    TimeSeriesOutcome('terminal_capacity_expanded_current', function=mean_over_replications),     
                    TimeSeriesOutcome('terminal_capacity_initial_current', function=mean_over_replications),      
                    TimeSeriesOutcome('terminal_denied_infra_cap_current', function=mean_over_replications),      
                    TimeSeriesOutcome('terminal_surface_current', function=mean_over_replications),               
                    TimeSeriesOutcome('TP_without_terminal_current', function=mean_over_replications),            
                    TimeSeriesOutcome('total_TP_rdam_current', function=mean_over_replications),                  
                    TimeSeriesOutcome('total_surface_terminals_current', function=mean_over_replications),          
                    TimeSeriesOutcome('business_case_terminal_current', function=mean_over_replications),         
                    TimeSeriesOutcome('business_value_pora_current', function=mean_over_replications),            
                    TimeSeriesOutcome('area_capacity_project', function=mean_over_replications),                  
                    TimeSeriesOutcome('area_capacity_unused_project', function=mean_over_replications),           
                    TimeSeriesOutcome('area_TP_project', function=mean_over_replications),                        
                    TimeSeriesOutcome('area_occupancy_project', function=mean_over_replications),                 
                    TimeSeriesOutcome('area_denied_infra_cap_project', function=mean_over_replications),          
                    TimeSeriesOutcome('area_denied_cap_percentage_project', function=mean_over_replications),     
                    TimeSeriesOutcome('terminal_capacity_project', function=mean_over_replications),              
                    TimeSeriesOutcome('terminal_capacity_unused_project', function=mean_over_replications),       
                    TimeSeriesOutcome('terminal_TP_project', function=mean_over_replications),                    
                    TimeSeriesOutcome('terminal_occupancy_project', function=mean_over_replications),             
                    TimeSeriesOutcome('terminal_capacity_expanded_project', function=mean_over_replications),     
                    TimeSeriesOutcome('terminal_capacity_initial_project', function=mean_over_replications),      
                    TimeSeriesOutcome('terminal_denied_infra_cap_project', function=mean_over_replications),      
                    TimeSeriesOutcome('terminal_surface_project', function=mean_over_replications),               
                    TimeSeriesOutcome('TP_without_terminal_project', function=mean_over_replications),            
                    TimeSeriesOutcome('total_TP_rdam_project', function=mean_over_replications),                  
                    TimeSeriesOutcome('total_surface_terminals_project', function=mean_over_replications),  
                    TimeSeriesOutcome('business_case_terminal_project', function=mean_over_replications),
                    TimeSeriesOutcome('business_value_pora_project', function=mean_over_replications),
                    TimeSeriesOutcome('coal_throughput_Rdam_project', function=mean_over_replications),
                    TimeSeriesOutcome('market_share_Rdam_project', function=mean_over_replications),
                    TimeSeriesOutcome('coal_throughput_Rdam_current', function=mean_over_replications),
                    TimeSeriesOutcome('market_share_Rdam_current', function=mean_over_replications),
                    ScalarOutcome('NPV', function=mean_over_replications)]

# specify model constants
model.constants = [Constant('run_times', 40),
                  Constant('project_to_assess','Infrastructure'),
                  Constant('energy_demand_init', 240000000),
                  Constant('other_grey_share_init', 0.54),
                  Constant('green_share_init', 0.267),
                  Constant('market_share_init', 0.376)
                  ]


In [2]:
with MultiprocessingEvaluator(model) as evaluator:
    results = evaluator.perform_experiments(1000)
save_results(results, r'ResultsEcosystem1109 rep 50 exp 10000.tar.gz')